In [19]:
import os
pwd = os.getcwd()

# Paths to sources
blaspp_source = "/home/weslleyp/storage/blaspp"
lapackpp_source = "/home/weslleyp/storage/lapackpp"
tlapack_source = "/home/weslleyp/storage/tlapack"

# Paths to libraries
tlapack_DIR = pwd+"/tlapack"
tlapackLAPACK_DIR = pwd+"/tlapack_LAPACK"
blaspp_DIR = pwd+"/blaspp"
lapackpp_DIR = pwd+"/lapackpp"
starpu_DIR = "/home/weslleyp/storage/starpu/build_install/lib"

# Set environment variables for StarPU
os.environ["PKG_CONFIG_PATH"] = starpu_DIR+"/pkgconfig"
os.environ["STARPU_SCHED"] = "dmdas"
os.environ["HWLOC_COMPONENTS"] = "-gl"

# Load Python modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import display, Math
import time

In [20]:
# System:
!uname -a

Linux weslleyp-XPS-15-9510 5.15.0-71-generic #78~20.04.1-Ubuntu SMP Wed Apr 19 11:26:48 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


In [21]:
# Machine:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   39 bits physical, 48 bits virtual
CPU(s):                          16
On-line CPU(s) list:             0-15
Thread(s) per core:              2
Core(s) per socket:              8
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           141
Model name:                      11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz
Stepping:                        1
CPU MHz:                         2300.000
CPU max MHz:                     4600.0000
CPU min MHz:                     800.0000
BogoMIPS:                        4608.00
Virtualization:                  VT-x
L1d cache:                       384 KiB
L1i cache:                       256 KiB
L2 cache:                        10 MiB
L3 cache:              

In [22]:
# Nvidia version:
!nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


In [23]:
# MKL version:
!which mkl_link_tool

/opt/intel/oneapi/mkl/2023.1.0/bin/intel64/mkl_link_tool


In [24]:
#Build

# Install <T>LAPACK
!cmake -B "$tlapack_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D BUILD_EXAMPLES=OFF -D BUILD_TESTING=OFF -D TLAPACK_NDEBUG=ON -D CMAKE_INSTALL_PREFIX="$tlapack_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" "$tlapack_source"
!cmake --build "$tlapack_DIR" --target install

# Build
!cmake -B build -G Ninja -D CMAKE_BUILD_TYPE=Release -D CMAKE_PREFIX_PATH="."
!cmake --build build

-- <T>LAPACK version 0.1.1
-- Submodule update
-- ClangFormat found: /usr/bin/clang-format-10 (version 10.0.0)
-- Configuring done
-- Generating done
-- Build files have been written to: /home/weslleyp/storage/tlapack/examples/starpu/tlapack
[0/1] Install the project...
-- Install configuration: "Release"
-- Installing: /home/weslleyp/storage/tlapack/examples/starpu/tlapack/include/tlapack/starpu/Matrix.hpp
-- Installing: /home/weslleyp/storage/tlapack/examples/starpu/tlapack/include/tlapack/starpu/tasks.hpp
-- A library with LAPACK API found.
-- A cache variable, namely LAPACKE_DIR, has been set to specify the install directory of LAPACKE
-- Looking for LAPACKE_dgeqrf
-- Looking for LAPACKE_dgeqrf - found
-- Looking for lapacke: test with lapack succeeds
-- Configuring done
-- Generating done
-- Build files have been written to: /home/weslleyp/storage/tlapack/examples/starpu/build
[6/6] Linking CXX executable example_starpu_potrftrf.dir/example_potrf.cpp.o


In [26]:
!./build/example_starpu_potrf 1000 50 all yes

n = 1000
nx = 50


-----------------------------------------------
float:
-----------------------------------------------
A = starpu::Matrix<f>( nrows = 1000, ncols = 1000 )
B = starpu::Matrix<f>( nrows = 1000, ncols = 1000 )
A = starpu::Matrix<f>( nrows = 1000, ncols = 1000 )
U^H U R = A   =>   ||R-Id||_1 / ||Id||_1 = 3.33709e-10
time = 0.0560023 s

-----------------------------------------------
double:
-----------------------------------------------
A = starpu::Matrix<d>( nrows = 1000, ncols = 1000 )
B = starpu::Matrix<d>( nrows = 1000, ncols = 1000 )
A = starpu::Matrix<d>( nrows = 1000, ncols = 1000 )
U^H U R = A   =>   ||R-Id||_1 / ||Id||_1 = 6.2788e-19
time = 0.0631591 s

-----------------------------------------------
complex<float>:
-----------------------------------------------
A = starpu::Matrix<St7complexIfE>( nrows = 1000, ncols = 1000 )
B = starpu::Matrix<St7complexIfE>( nrows = 1000, ncols = 1000 )
A = starpu::Matrix<St7complexIfE>( nrows = 1000, ncols = 1000 )
U^H U R =

In [25]:
!./build/example_starpu_lapack 1000

n = 1000

-----------------------------------------------
float:
-----------------------------------------------
time = 0.0462456 s

-----------------------------------------------
double:
-----------------------------------------------
time = 0.00361736 s

-----------------------------------------------
complex<float>:
-----------------------------------------------
time = 0.003967 s

-----------------------------------------------
complex<double>:
-----------------------------------------------
time = 0.00742062 s


In [27]:
#Build with MKL

# Install BLAS++
!cmake -B "$blaspp_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D build_tests=OFF -D CMAKE_INSTALL_PREFIX="$blaspp_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" "$blaspp_source"
!cmake --build "$blaspp_DIR" --target install

# Install LAPACK++
!cmake -B "$lapackpp_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D build_tests=OFF -D CMAKE_INSTALL_PREFIX="$lapackpp_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" -D blaspp_DIR="$blaspp_DIR" "$lapackpp_source"
!cmake --build "$lapackpp_DIR" --target install

# Install <T>LAPACK
!cmake -B "$tlapackLAPACK_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D BUILD_EXAMPLES=OFF -D BUILD_TESTING=OFF -D TLAPACK_NDEBUG=ON -D CMAKE_INSTALL_PREFIX="$tlapackLAPACK_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" -D USE_LAPACKPP_WRAPPERS=ON -D blaspp_DIR="$blaspp_DIR" -D lapackpp_DIR="$lapackpp_DIR" "$tlapack_source"
!cmake --build "$tlapackLAPACK_DIR" --target install

# Build
!cmake -B build_LAPACK -G Ninja -D CMAKE_BUILD_TYPE=Release -D tlapack_DIR="$tlapackLAPACK_DIR" -D blaspp_DIR="$blaspp_DIR" -D lapackpp_DIR="$lapackpp_DIR"
!cmake --build build_LAPACK

-- Using CMAKE_INSTALL_PREFIX = /home/weslleyp/storage/tlapack/examples/starpu/blaspp
-- Not building CUDA support in BLAS++
-- Looking for HIP/ROCm
-- Not building HIP/ROCm support in BLAS++
-- blaspp_id = c498e8d
-- Configuring done
-- Generating done
-- Build files have been written to: /home/weslleyp/storage/tlapack/examples/starpu/blaspp
[0/1] Install the project...
-- Install configuration: "Release"
-- Installing: /home/weslleyp/storage/tlapack/examples/starpu/blaspp/lib/blaspp/blasppTargets-release.cmake
-- Using CMAKE_INSTALL_PREFIX = /home/weslleyp/storage/tlapack/examples/starpu/lapackpp
-- Not building CUDA support in LAPACK++
-- Looking for HIP/ROCm
-- Not building HIP/ROCm support in LAPACK++
-- lapackpp_id = 17edf95
-- Check for BLAS++
   Found BLAS++: /home/weslleyp/storage/tlapack/examples/starpu/blaspp
-- Configuring done
-- Generating done
-- Build files have been written to: /home/weslleyp/storage/tlapack/examples/starpu/lapackpp
[0/1] Install the project...
-- Inst

In [28]:
!./build_LAPACK/example_starpu_potrf 1000 50 all yes

n = 1000
nx = 50


-----------------------------------------------
float:
-----------------------------------------------
A = starpu::Matrix<f>( nrows = 1000, ncols = 1000 )
B = starpu::Matrix<f>( nrows = 1000, ncols = 1000 )
A = starpu::Matrix<f>( nrows = 1000, ncols = 1000 )
U^H U R = A   =>   ||R-Id||_1 / ||Id||_1 = 3.31816e-10
time = 0.0613892 s

-----------------------------------------------
double:
-----------------------------------------------
A = starpu::Matrix<d>( nrows = 1000, ncols = 1000 )
B = starpu::Matrix<d>( nrows = 1000, ncols = 1000 )
A = starpu::Matrix<d>( nrows = 1000, ncols = 1000 )
U^H U R = A   =>   ||R-Id||_1 / ||Id||_1 = 6.51526e-19
time = 0.0973496 s

-----------------------------------------------
complex<float>:
-----------------------------------------------
A = starpu::Matrix<St7complexIfE>( nrows = 1000, ncols = 1000 )
B = starpu::Matrix<St7complexIfE>( nrows = 1000, ncols = 1000 )
A = starpu::Matrix<St7complexIfE>( nrows = 1000, ncols = 1000 )
U^H U R 

# First test: Find optimal block size for a given matrix size

In [11]:
# Set environment variables for StarPU
if 'STARPU_NCUDA' in os.environ:
    del os.environ['STARPU_NCUDA']
if 'STARPU_NCPU' in os.environ:
    del os.environ['STARPU_NCPU']

In [12]:
n = 7680

# vector with integer factors of n
parts = [i for i in range(8,21) if n%i==0]
N = len(parts)

precision = ["single","double"]
NT = len(precision)

nRuns = 5

executable = [
    "build_LAPACK/example_starpu_potrf"
]
methods = [
    r"C++ templates + cuBLAS + cuSOLVER + MKL"
]
M = len(executable)

# Partition sizes
nb = np.divide(n,parts)

# FLOPs
FLOPs_mul = n * (((n*1.)/6+.5)*n+1./3)
FLOPs_add = n * (((n*1.)/6)*n-1./6)
FLOPs = FLOPs_mul + FLOPs_add

print("Size of the matrix = "+str(n))
print("Partitions: "+str(parts))
print("Precisions: "+str(precision))
print("Number of runs = "+str(nRuns))

Size of the matrix = 7680
Partitions: [8, 10, 12, 15, 16, 20]
Precisions: ['single', 'double']
Number of runs = 5


In [13]:
expr = executable[0]
!$expr {n} {parts[0]} {precision[0]}

n = 7680
nx = 8


-----------------------------------------------
float:
-----------------------------------------------
A = starpu::Matrix<f>( nrows = 7680, ncols = 7680 )


In [14]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

data = np.ones([M,N,NT], dtype=np.float64) * 60 * 60 * 24

for s in range(M):
    expr = executable[s]
    for i in range(N):
        nx = parts[i]
        for k in range(NT):
            for j in range(nRuns):
                time.sleep(5)
                output = !$expr {n} {nx} {precision[k]} | grep time
                print(output)
                try:
                    aux = float(output[-1].split()[2])
                    data[s,i,k] = np.minimum( aux, data[s,i,k] )
                except:
                    print("Not a float: ", output)

[]
Not a float:  []


KeyboardInterrupt: 

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for p in range(NT):
    print(precision[p])

    fig1, ax1 = plt.subplots()

    for m in range(M):
        gflops = np.divide(FLOPs/1e9,data[m,:,p])
        plt.plot(nb,gflops,markers[m%3],label=methods[m])

    # ax1.set_xscale("log")
    # ax1.set_yscale("log")
    # ax1.set_xticks(parts)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("$n_b$")
    plt.ylabel("GFLOPS")
    plt.legend()

    plt.tight_layout()
    plt.savefig("curves_"+precision[p]+".pdf")
    plt.show()

# Take the best block size and compare with MKL and StarPU

In [ ]:
nx_best = np.zeros([NT], dtype=np.float64)
for k in range(NT):
    nx_best[k] = parts[np.argmin(data[0,:,k])]

print(nx_best)

nRuns = 5 # for the best partition size

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

dataMKL = np.ones([NT], dtype=np.float64) * 60 * 60 * 24

expr = "build/example_starpu_lapack"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr {n} {precision[k]} | grep time
        print(output)
        try:
            aux = float(output[-1].split()[2])
            dataMKL[k] = np.minimum( aux, dataMKL[k] )
        except:
            print("Not a float: ", output)
#         time.sleep(5)

for p in range(NT):
    print(precision[p])

    gflops = np.divide(FLOPs/1e9,dataMKL[p])
    print("time[s] = ", dataMKL[p])
    print("GFLOPS = ", gflops)

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

dataTLAPACKStarPUmkl = np.ones([NT], dtype=np.float64) * 60 * 60 * 24

expr = "build_LAPACK/example_starpu_potrf"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr {n} {nx_best[k]} {precision[k]} | grep time
        print(output)
        try:
            aux = float(output[-1].split()[2])
            dataTLAPACKStarPUmkl[k] = np.minimum( aux, dataTLAPACKStarPUmkl[k] )
        except:
            print("Not a float: ", output)

for p in range(NT):
    print(precision[p])

    gflops = np.divide(FLOPs/1e9,dataTLAPACKStarPUmkl[p])
    print("time[s] = ", dataTLAPACKStarPUmkl[p])
    print("GFLOPS = ", gflops)

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

expr = starpu_DIR + "/starpu/examples/cholesky_implicit"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr
        print(output[-1])

# First test: Find optimal block size without GPU

In [ ]:
# Set environment variables for StarPU
os.environ["STARPU_NCUDA"] = "0"
if 'STARPU_NCPU' in os.environ:
    del os.environ['STARPU_NCPU']

In [ ]:
n = 7680

# vector with integer factors of n
parts = [i for i in range(6,60) if n%i==0]
N = len(parts)

precision = ["single"]
NT = len(precision)

nRuns = 5

executable = [
    "build_LAPACK/example_starpu_potrf"
]
methods = [
    r"C++ templates + MKL"
]
M = len(executable)

# Partition sizes
nb = np.divide(n,parts)

# FLOPs
FLOPs_mul = n * (((n*1.)/6+.5)*n+1./3)
FLOPs_add = n * (((n*1.)/6)*n-1./6)
FLOPs = FLOPs_mul + FLOPs_add

print("Size of the matrix = "+str(n))
print("Partitions: "+str(parts))
print("Precisions: "+str(precision))
print("Number of runs = "+str(nRuns))

In [ ]:
expr = executable[0]
!$expr {n} {parts[0]} {precision[0]} y

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

data = np.ones([M,N,NT], dtype=np.float64) * 60 * 60 * 24

for s in range(M):
    expr = executable[s]
    for i in range(N):
        nx = parts[i]
        for k in range(NT):
            for j in range(nRuns):
                time.sleep(5)
                output = !$expr {n} {nx} {precision[k]} | grep time
                print(output)
                try:
                    aux = float(output[-1].split()[2])
                    data[s,i,k] = np.minimum( aux, data[s,i,k] )
                except:
                    print("Not a float: ", output)

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for p in range(NT):
    print(precision[p])

    fig1, ax1 = plt.subplots()

    for m in range(M):
        gflops = np.divide(FLOPs/1e9,data[m,:,p])
        plt.plot(nb,gflops,markers[m%3],label=methods[m])

    # ax1.set_xscale("log")
    # ax1.set_yscale("log")
    # ax1.set_xticks(parts)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("$n_b$")
    plt.ylabel("GFLOPS")
    plt.legend()

    plt.tight_layout()
    plt.savefig("curves_"+precision[p]+".pdf")
    plt.show()

In [ ]:
nx_best = np.zeros([NT], dtype=np.float64)
for k in range(NT):
    nx_best[k] = parts[np.argmin(data[0,:,k])]
nx_best[0] = 15
print("Best nb:", np.divide(n,nx_best))

nRuns = 5 # for the best partition size

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

dataTLAPACKStarPUmkl = np.ones([NT], dtype=np.float64) * 60 * 60 * 24

expr = "build_LAPACK/example_starpu_potrf"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr {n} {nx_best[k]} {precision[k]} | grep time
        print(output)
        try:
            aux = float(output[-1].split()[2])
            dataTLAPACKStarPUmkl[k] = np.minimum( aux, dataTLAPACKStarPUmkl[k] )
        except:
            print("Not a float: ", output)

for p in range(NT):
    print(precision[p])

    gflops = np.divide(FLOPs/1e9,dataTLAPACKStarPUmkl[p])
    print("time[s] = ", dataTLAPACKStarPUmkl[p])
    print("GFLOPS = ", gflops)

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

expr = starpu_DIR + "/starpu/examples/cholesky_implicit"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr
        print(output[-1])

In [ ]:
expr = "build_LAPACK/example_starpu_potrf"
!$expr {n} {n/48} single